<a href="https://colab.research.google.com/github/HighLvRiver/HighLvRiver.github.io/blob/master/AIC_Project1_Jayden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [0]:
train = pd.read_csv("https://www.dropbox.com/s/hp2dn6l6op4n4mv/watcha_aic_train_data_for_problem_1.csv?dl=1")
train.head(10)

,userId,movieId,preference
0,31231,1193,1.0
1,32077,1284,1.0
2,37618,368,0.0
3,76002,490,1.0
4,79452,2132,0.0
5,128354,2542,0.0
6,68571,190,0.0
7,107371,21,0.0
8,12480,1357,-1.0
9,51994,380,1.0


In [0]:
test = pd.read_csv("https://www.dropbox.com/s/0g70q9k7pm5so1n/watcha_aic_test_data_for_problem_1.csv?dl=1")
test.head(10)

,userId,movieId,preference
0,118853,4232,0.0
1,1734,1701,1.0
2,65850,535,0.0
3,99208,6832,1.0
4,133444,318,1.0
5,36468,3683,1.0
6,78722,1834,0.0
7,9720,262,0.0
8,120385,296,-1.0
9,5348,25,1.0


In [0]:
test.shape

(3992967, 3)

In [0]:
train.shape

(15971866, 3)

In [0]:
# 최종 점수 산정 함수 

def score(ans, pred):
    if ans == 0: 
      return 0
    elif ans == pred: 
      return 1
    else: 
      return -1

In [0]:
# 최대 점수 체크하기 : 2,973,129

test.apply(lambda s: score(s['preference'],s['preference']), axis=1).sum() # 2973129

2973129

In [0]:
prefer = train # train[:10000]
prefer_matrix = prefer.pivot_table('preference',index='userId', columns='movieId')
prefer_matrix.head(5)

movieId,1,2,3,5,6,7,9,10,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,29,30,31,32,34,35,36,39,40,41,42,43,44,45,46,47,...,102720,103042,103141,103277,103335,103339,103655,103772,103810,104272,104374,104457,105213,106487,106489,106696,106916,106920,107406,108571,108932,109072,109374,109487,109848,110771,111384,111443,111759,112138,112421,112556,113064,114246,114795,115149,115569,118706,119145,119153
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
  num_users, num_items = R.shape
  # P와 Q 매트릭스의 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력
  np.random.seed(1)
  P = np.random.normal(scale=1./K, size=(num_users, K))
  Q = np.random.normal(scale=1./K, size=(num_items, K))

  prev_rmse = 10000
  break_count = 0

  # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장
  non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

  # SGD기법으로 P와 Q 매트릭스를 계속 업데이트
  for step in range(steps):
    for i, j ,r in non_zeros:
      # 실제 값과 예측 값의 차이인 오류 값 구함
      eij = r - np.dot(P[i,:], Q[j,:].T)
      # Regularization을 반영한 SGD 업데이트 공식 적용
      P[i,:] = P[i,:] + learning_rate*(eij * Q[j,:] - r_lambda*P[i,:])
      Q[j,:] = Q[j,:] + learning_rate*(eij * P[i,:] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 10) == 0:
      print("### iteration step : ", step, " rmse : ", rmse)
    
  return P, Q

In [0]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
  
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [0]:
# matrix_factorization() 함수를 이용해 행렬 분해
# 수행 시간이 오래 걸리므로 SGD 반복 횟수인 steps는 200회만 지정
# 잠재요인 차원 K는 50, 학습률과 L2 Regularization 계수는 모두 0.01로 설정하고 수행
start = time.time()
P, Q = matrix_factorization(prefer_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda=0.01)
pred_matrix = np.dot(P, Q.T)
print('Training time(s) :'+str(int(time.time() - start)))

### iteration step :  0  rmse :  0.9995636634852464
### iteration step :  10  rmse :  0.9953520690173295
### iteration step :  20  rmse :  0.989934135857466
### iteration step :  30  rmse :  0.981648124988215
### iteration step :  40  rmse :  0.9677608835157585
### iteration step :  50  rmse :  0.9445648141931118
### iteration step :  60  rmse :  0.9098755664229531
### iteration step :  70  rmse :  0.8659647572468514
### iteration step :  80  rmse :  0.817056135305029
### iteration step :  90  rmse :  0.7662955122088702
### iteration step :  100  rmse :  0.7156186197647443
### iteration step :  110  rmse :  0.6661101194610612
### iteration step :  120  rmse :  0.6183076362100345
### iteration step :  130  rmse :  0.572454784685647
### iteration step :  140  rmse :  0.528725725090381
### iteration step :  150  rmse :  0.48729995981094704
### iteration step :  160  rmse :  0.4483003294836611
### iteration step :  170  rmse :  0.41172690879888807
### iteration step :  180  rmse :  0.37745

In [0]:
prefer_pred_matrix = pd.DataFrame(data=pred_matrix, index=prefer_matrix.index, columns=prefer_matrix.columns)

prefer_pred_matrix.head(3)

movieId,1,2,3,5,6,7,9,10,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,29,30,31,32,34,35,36,39,40,41,42,43,44,45,46,47,...,102720,103042,103141,103277,103335,103339,103655,103772,103810,104272,104374,104457,105213,106487,106489,106696,106916,106920,107406,108571,108932,109072,109374,109487,109848,110771,111384,111443,111759,112138,112421,112556,113064,114246,114795,115149,115569,118706,119145,119153
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
24,0.009653,-0.019776,-0.001561,0.009984,0.008979,0.016222,0.000118,0.020827,0.029514,0.002202,-0.010987,-0.002274,-0.010869,-0.009063,0.001596,-0.010066,-0.000991,-0.041350,0.001508,0.002767,-0.017344,-0.026321,0.003720,-0.011533,0.050155,-0.005101,0.042560,0.007751,0.019623,-0.000235,-0.024953,-0.030355,0.014609,0.000310,-0.009248,-0.022705,0.010456,-0.028069,-0.001683,-0.016814,...,0.000804,-0.006263,-0.000345,0.000930,-0.027311,-0.001369,-0.002817,0.004463,0.004710,-0.003104,-0.009755,-0.000451,0.010741,-0.012182,0.002123,-0.008003,-0.002327,-0.003844,-0.014367,0.000604,0.015889,-0.002580,0.000185,-0.001663,-0.004804,-0.000113,0.017145,0.007338,0.000501,0.002059,0.000755,0.025591,0.004595,0.027029,-0.000348,0.009250,-0.000240,0.007974,-0.014130,0.015968
25,0.029208,-0.002390,0.140576,0.097366,-0.024417,0.079924,0.031791,0.013181,-0.006834,-0.016532,0.069342,0.007911,-0.142861,0.074322,0.001111,-0.019577,-0.000045,0.227586,-0.019099,0.008878,0.038303,0.034215,0.001681,-0.017657,0.059977,-0.090274,0.045799,-0.181338,0.145785,-0.003349,-0.048174,0.037424,0.023609,0.010662,-0.063419,0.042439,-0.066404,-0.054300,-0.012078,0.134287,...,0.007918,-0.051033,0.006047,-0.001053,-0.059494,0.008551,0.002209,0.008056,-0.000483,-0.022860,-0.086234,-0.006289,0.053166,0.019987,0.062474,0.108826,0.006399,0.014896,0.056362,-0.006998,-0.040557,0.001523,-0.029621,-0.004979,-0.009788,-0.018531,-0.092369,0.081912,-0.007021,0.001677,0.028970,-0.047545,-0.059851,0.043328,0.012499,0.003137,-0.030062,0.037213,0.003560,0.015959
35,0.045839,0.240600,-0.119322,-0.036916,-0.052094,-0.008415,-0.046070,-0.130038,0.042590,0.026573,-0.064874,-0.012263,0.037921,-0.043763,-0.002546,0.063558,-0.007996,-0.051785,-0.001487,-0.002573,0.057898,-0.008185,-0.005170,0.056812,0.039646,0.145969,0.113710,0.111901,-0.095182,-0.011899,0.103846,0.156180,-0.037150,-0.020708,0.032774,-0.017125,0.038720,-0.070788,0.006471,-0.187378,...,-0.002288,0.055417,-0.012531,-0.021484,0.130205,-0.018564,0.010711,-0.004271,-0.039340,0.008955,0.019889,0.008052,-0.076634,0.148841,-0.087234,-0.013029,0.016344,0.017234,-0.043920,-0.007886,-0.068038,0.012542,0.108241,-0.012120,-0.011969,0.007171,0.173469,-0.019075,-0.053976,0.007022,-0.011754,-0.139270,-0.009970,-0.063230,0.016881,-0.072535,0.013530,-0.006287,-0.045953,-0.053171
